# Final Project
---
## Mofidul Islam Jamal
##Web 2.0 Architectures and Algorithms
##Spring 2015
---

## Option 2, Education Data
For my assignment, I have opted to use option 2 which involves using 2013 college graduate enrollment information. I will parse the information provided as per the instructions, and display maps using the folium library on a per state basis.

I have discovered that a naive way of showing enrollment data per ethnicity group does not provide very much information, so I have also opted to show the enrollment data per ethnicity group localized to the state.

The set of graphs I show are:
* % of each ethnicity group in each state as a percentage of the total for that ethnicity group within the US
 * For example, if you see 5% in the state of california for ethnicity group Asian, this means that 5% of all enrolled asians in the US that enrollment data is available for exist in california.
* % of each ethnicity group in each state as a percentage of the total for that ethnicity group 
 * For example, if you see 5% in the state of california for ethnicity group Asian, this means that 5% of all enrolled students in california that data is available for is asian
 
The maps have a bit of a data bias, as the maps which are relative to the US almost always have larger #'s in the states with the highest % of the population.

The maps that are relative to each individual state are much more 
 
This notebook has depency on the following, installable via pip:
folium
pretty

### Load the data
This section will load the data from the two databases, the enrollment data and the index data for the schools. Additionally the enrollment data is filtered to only graduates (EFALEVEL of 12)

Following that, the school index data will be parsed to build a mapping between the school UNITID in the enrollment data and the state abbreviation for that school. This allows us to determine which state a school belongs to.

Then finally we have a block to zero out our data values

In [22]:
#we skip total aliens
import csv
from pretty import pprint

enrollmentdata = open('ef2013a.csv') 
schooldata = open('hd2013.csv')
graduate_data = []
school_data = []
enrollment_reader = csv.DictReader(enrollmentdata)
school_reader = csv.DictReader(schooldata)

#first get all the entries for the schools that contain graduate data
for record in enrollment_reader:
    if record['EFALEVEL'] == '12':
        graduate_data.append(record)

#next get entries for the school info
for record in school_reader:
    school_data.append(record)

#close the file handles
enrollmentdata.close()
schooldata.close()

#at this point we have all the data that we need, so we can begin analysis
print "there are {} records of graduate school data".format(len(graduate_data))
print "there are {} school records".format(len(school_data))

there are 2167 records of graduate school data
there are 7764 school records


In [23]:
#strip out unncesary fields by reconstructing the school data table
#we will map the UNITID to the state abbreviation
unitid_to_state = {}
for record in school_data:
    unitid_to_state[record['UNITID']] = record['STABBR']

In [24]:
#store data per state in the following
data_per_state = {}
#we need to blank these out
data_total = {'totalall' : 0,
            'totalmen' : 0,
            'totalwomen' : 0,
            'totalindian' : 0,
            'totalindianmen' : 0,
            'totalindianwomen' : 0,
            'totalasian' : 0,
            'totalasianmen' : 0,
            'totalasianwomen' : 0,
            'totalblack' : 0,
            'totalblackmen' : 0,
            'totalblackwomen' : 0,
            'totalhispanic' : 0,
            'totalhispanicmen' : 0,
            'totalhispanicwomen' : 0,
            'totalislander' : 0,
            'totalislandermen' : 0,
            'totalislanderwomen' : 0,
            'totalwhite' : 0,
            'totalwhitemen' : 0,
            'totalwhitewomen' : 0,
            'totalbiracial' : 0,
            'totalbiracialmen' : 0,
            'totalbiracialwomen' : 0,
            'totalunknown' : 0,
            'totalunknownmen' : 0,
            'totalunknownwomen' : 0,
              }

### Parsing the records
This block of code will parse through all the records and increment the total counts for the US for all ethnicities and increment the total counts per state using the lookup table created earlier.

In [25]:
#next we will now go through all the records, parse out the state name and look it up and then tally up the totals for the states
error_count = 0 #keeps track of the # of records we failed to parse
error_records = [] #keeps track of all the error records
for record in graduate_data:
    state = unitid_to_state[record['UNITID']] #get state from school
    #now that we have the state, lets get the values
    totalall =              int(record['EFTOTLT'])
    totalmen =              int(record['EFTOTLM'])
    totalwomen =            int(record['EFTOTLW'])
    totalindian =           int(record['EFAIANT'])
    totalindianmen =        int(record['EFAIANM'])
    totalindianwomen =      int(record['EFAIANW'])
    totalasian =            int(record['EFASIAT'])
    totalasianmen =         int(record['EFASIAM'])
    totalasianwomen =       int(record['EFASIAW'])
    totalblack =            int(record['EFBKAAT'])
    totalblackmen =         int(record['EFBKAAM'])
    totalblackwomen =       int(record['EFBKAAW'])
    totalhispanic =         int(record['EFHISPT'])
    totalhispanicmen =      int(record['EFHISPM'])
    totalhispanicwomen =    int(record['EFHISPW'])
    totalislander =         int(record['EFNHPIT'])
    totalislandermen =      int(record['EFNHPIM'])
    totalislanderwomen =    int(record['EFNHPIW'])
    totalwhite =            int(record['EFWHITT'])
    totalwhitemen =         int(record['EFWHITM'])
    totalwhitewomen =       int(record['EFWHITW'])
    totalbiracial =         int(record['EF2MORT'])
    totalbiracialmen =      int(record['EF2MORM'])
    totalbiracialwomen =    int(record['EF2MORW'])
    totalunknown =          int(record['EFUNKNT'])
    totalunknownmen =       int(record['EFUNKNM'])
    totalunknownwomen =     int(record['EFUNKNW'])
    #tabulate the grand total for all state
    data_total['totalall'] += totalall
    data_total['totalmen'] += totalmen
    data_total['totalwomen'] += totalwomen
    data_total['totalindian'] += totalindian
    data_total['totalindianmen'] += totalindianmen
    data_total['totalindianwomen'] += totalindianwomen
    data_total['totalasian'] += totalasian
    data_total['totalasianmen'] += totalasianmen
    data_total['totalasianwomen'] += totalasianwomen
    data_total['totalblack'] += totalblack
    data_total['totalblackmen'] += totalblackmen
    data_total['totalblackwomen'] += totalblackwomen
    data_total['totalhispanic'] += totalhispanic
    data_total['totalhispanicmen'] += totalhispanicmen
    data_total['totalhispanicwomen'] += totalhispanicwomen
    data_total['totalislander'] += totalislander
    data_total['totalislandermen'] += totalislandermen
    data_total['totalislanderwomen'] += totalislanderwomen
    data_total['totalwhite'] += totalwhite
    data_total['totalwhitemen'] += totalwhitemen
    data_total['totalwhitewomen'] += totalwhitewomen
    data_total['totalbiracial'] += totalbiracial
    data_total['totalbiracialmen'] += totalbiracialmen
    data_total['totalbiracialwomen'] += totalbiracialwomen
    data_total['totalunknown'] += totalunknown
    data_total['totalunknownmen'] += totalunknownmen
    data_total['totalunknownwomen'] += totalunknownwomen
    #now add them to the state totals and fips totals
    try:
        #state totals
        if state in data_per_state.keys(): # key in dict so we can add
            data_per_state[state]['totalall'] += totalall
            data_per_state[state]['totalmen'] += totalmen
            data_per_state[state]['totalwomen'] += totalwomen
            data_per_state[state]['totalindian'] += totalindian
            data_per_state[state]['totalindianmen'] += totalindianmen
            data_per_state[state]['totalindianwomen'] += totalindianwomen
            data_per_state[state]['totalasian'] += totalasian
            data_per_state[state]['totalasianmen'] += totalasianmen
            data_per_state[state]['totalasianwomen'] += totalasianwomen
            data_per_state[state]['totalblack'] += totalblack
            data_per_state[state]['totalblackmen'] += totalblackmen
            data_per_state[state]['totalblackwomen'] += totalblackwomen
            data_per_state[state]['totalhispanic'] += totalhispanic
            data_per_state[state]['totalhispanicmen'] += totalhispanicmen
            data_per_state[state]['totalhispanicwomen'] += totalhispanicwomen
            data_per_state[state]['totalislander'] += totalislander
            data_per_state[state]['totalislandermen'] += totalislandermen
            data_per_state[state]['totalislanderwomen'] += totalislanderwomen
            data_per_state[state]['totalwhite'] += totalwhite
            data_per_state[state]['totalwhitemen'] += totalwhitemen
            data_per_state[state]['totalwhitewomen'] += totalwhitewomen
            data_per_state[state]['totalbiracial'] += totalbiracial
            data_per_state[state]['totalbiracialmen'] += totalbiracialmen
            data_per_state[state]['totalbiracialwomen'] += totalbiracialwomen
            data_per_state[state]['totalunknown'] += totalunknown
            data_per_state[state]['totalunknownmen'] += totalunknownmen
            data_per_state[state]['totalunknownwomen'] += totalunknownwomen
        else: #key is not in the dict so that means this is a state without any entry, we must assign
            data_per_state[state] = {'state' : state,
                                    'totalall' : totalall,
                                    'totalmen' : totalmen,
                                    'totalwomen' : totalwomen,
                                    'totalindian' : totalindian,
                                    'totalindianmen' : totalindianmen,
                                    'totalindianwomen' : totalindianwomen,
                                    'totalasian' : totalasian,
                                    'totalasianmen' : totalasianmen,
                                    'totalasianwomen' : totalasianwomen,
                                    'totalblack' : totalblack,
                                    'totalblackmen' : totalblackmen,
                                    'totalblackwomen' : totalblackwomen,
                                    'totalhispanic' : totalhispanic,
                                    'totalhispanicmen' : totalhispanicmen,
                                    'totalhispanicwomen' : totalhispanicwomen,
                                    'totalislander' : totalislander,
                                    'totalislandermen' : totalislandermen,
                                    'totalislanderwomen' : totalislanderwomen,
                                    'totalwhite' : totalwhite,
                                    'totalwhitemen' : totalwhitemen,
                                    'totalwhitewomen' : totalwhitewomen,
                                    'totalbiracial' : totalbiracial,
                                    'totalbiracialmen' : totalbiracialmen,
                                    'totalbiracialwomen' : totalbiracialwomen,
                                    'totalunknown' : totalunknown,
                                    'totalunknownmen' : totalunknownmen,
                                    'totalunknownwomen' : totalunknownwomen,
                                    }
    except KeyError as e:
        error_count += 1 #increment error count

In [26]:
pprint(data_total)

{'totalasianwomen': 95630,
 'totalindianmen': 4909,
 'totalblack': 330709,
 'totalunknownwomen': 142083,
 'totalbiracialmen': 18904,
 'totalwhitewomen': 912223,
 'totalasian': 172526,
 'totalislander': 6295,
 'totalislandermen': 2522,
 'totalhispanicwomen': 143165,
 'totalwhite': 1547016,
 'totalwomen': 1725107,
 'totalhispanic': 228334,
 'totalunknown': 239321,
 'totalasianmen': 76896,
 'totalblackmen': 98485,
 'totalhispanicmen': 85169,
 'totalindianwomen': 8667,
 'totalunknownmen': 97238,
 'totalislanderwomen': 3773,
 'totalmen': 1221956,
 'totalbiracial': 49873,
 'totalwhitemen': 634793,
 'totalall': 2947063,
 'totalblackwomen': 232224,
 'totalindian': 13576,
 'totalbiracialwomen': 30969}


###Calculate the data per state relative to the US
This block will take all the state data and divide it by the totals for each ethnicity in the US. This will give us a number for each state which signifies the % of the total for that ethnicity in that state compared to the US total for that ethnicity

In [27]:
import copy
#we are going to calculate what % of each state corresponds to each category
#that is to say, we want to map what % of a state out of the total of all states falls into that category
#that way when we draw the map, the colors will be relative to the total
#this is better than drawing the map using the absolute numbers because there is a tendency for
#the absolute enrollment numbers to hide info because the states with the highest enrollmen or 
#states with the most people will always be colored more brightly
data_per_state_relative_to_us = copy.deepcopy(data_per_state)
for state in data_per_state_relative_to_us.keys():
    data_per_state_relative_to_us[state]['totalall'] = (float(data_per_state_relative_to_us[state]['totalall']) / data_total['totalall']) * 100.0 
    data_per_state_relative_to_us[state]['totalmen'] = (float(data_per_state_relative_to_us[state]['totalmen']) / data_total['totalmen']) * 100.0 
    data_per_state_relative_to_us[state]['totalwomen'] = (float(data_per_state_relative_to_us[state]['totalwomen']) / data_total['totalwomen']) * 100.0 
    data_per_state_relative_to_us[state]['totalindian'] = (float(data_per_state_relative_to_us[state]['totalindian']) / data_total['totalindian']) * 100.0 
    data_per_state_relative_to_us[state]['totalindianmen'] = (float(data_per_state_relative_to_us[state]['totalindianmen']) / data_total['totalindianmen']) * 100.0 
    data_per_state_relative_to_us[state]['totalindianwomen'] = (float(data_per_state_relative_to_us[state]['totalindianwomen']) / data_total['totalindianwomen']) * 100.0 
    data_per_state_relative_to_us[state]['totalasian'] = (float(data_per_state_relative_to_us[state]['totalasian']) / data_total['totalasian']) * 100.0 
    data_per_state_relative_to_us[state]['totalasianmen'] = (float(data_per_state_relative_to_us[state]['totalasianmen']) / data_total['totalasianmen']) * 100.0 
    data_per_state_relative_to_us[state]['totalasianwomen'] = (float(data_per_state_relative_to_us[state]['totalasianwomen']) / data_total['totalasianwomen']) * 100.0 
    data_per_state_relative_to_us[state]['totalblack'] = (float(data_per_state_relative_to_us[state]['totalblack']) / data_total['totalblack']) * 100.0 
    data_per_state_relative_to_us[state]['totalblackmen'] = (float(data_per_state_relative_to_us[state]['totalblackmen']) / data_total['totalblackmen']) * 100.0 
    data_per_state_relative_to_us[state]['totalblackwomen'] = (float(data_per_state_relative_to_us[state]['totalblackwomen']) / data_total['totalblackwomen']) * 100.0 
    data_per_state_relative_to_us[state]['totalhispanic'] = (float(data_per_state_relative_to_us[state]['totalhispanic']) / data_total['totalhispanic']) * 100.0 
    data_per_state_relative_to_us[state]['totalhispanicmen'] = (float(data_per_state_relative_to_us[state]['totalhispanicmen']) / data_total['totalhispanicmen']) * 100.0 
    data_per_state_relative_to_us[state]['totalhispanicwomen'] = (float(data_per_state_relative_to_us[state]['totalhispanicwomen']) / data_total['totalhispanicwomen']) * 100.0 
    data_per_state_relative_to_us[state]['totalislander'] = (float(data_per_state_relative_to_us[state]['totalislander']) / data_total['totalislander']) * 100.0 
    data_per_state_relative_to_us[state]['totalislandermen'] = (float(data_per_state_relative_to_us[state]['totalislandermen']) / data_total['totalislandermen']) * 100.0 
    data_per_state_relative_to_us[state]['totalislanderwomen'] = (float(data_per_state_relative_to_us[state]['totalislanderwomen']) / data_total['totalislanderwomen']) * 100.0 
    data_per_state_relative_to_us[state]['totalwhite'] = (float(data_per_state_relative_to_us[state]['totalwhite']) / data_total['totalwhite']) * 100.0 
    data_per_state_relative_to_us[state]['totalwhitemen'] = (float(data_per_state_relative_to_us[state]['totalwhitemen']) / data_total['totalwhitemen']) * 100.0 
    data_per_state_relative_to_us[state]['totalwhitewomen'] = (float(data_per_state_relative_to_us[state]['totalwhitewomen']) / data_total['totalwhitewomen']) * 100.0 
    data_per_state_relative_to_us[state]['totalbiracial'] = (float(data_per_state_relative_to_us[state]['totalbiracial']) / data_total['totalbiracial']) * 100.0 
    data_per_state_relative_to_us[state]['totalbiracialmen'] = (float(data_per_state_relative_to_us[state]['totalbiracialmen']) / data_total['totalbiracialmen']) * 100.0 
    data_per_state_relative_to_us[state]['totalbiracialwomen'] = (float(data_per_state_relative_to_us[state]['totalbiracialwomen']) / data_total['totalbiracialwomen']) * 100.0 
    data_per_state_relative_to_us[state]['totalunknown'] = (float(data_per_state_relative_to_us[state]['totalunknown']) / data_total['totalunknown']) * 100.0 
    data_per_state_relative_to_us[state]['totalunknownmen'] = (float(data_per_state_relative_to_us[state]['totalunknownmen']) / data_total['totalunknownmen']) * 100.0 
    data_per_state_relative_to_us[state]['totalunknownwomen'] = (float(data_per_state_relative_to_us[state]['totalunknownwomen']) / data_total['totalunknownwomen']) * 100.0  
pprint(data_per_state_relative_to_us)

    

{'WA': {'totalasianwomen': 1.8069643417337653,
  'totalindianmen': 2.098187003463027,
  'totalblack': 0.3447139327928783,
  'totalunknownwomen': 1.0733163010353104,
  'totalbiracialmen': 2.1688531527719,
  'totalwhitewomen': 1.3664421966997105,
  'totalasian': 1.713364942095684,
  'totalislander': 1.5091342335186657,
  'totalislandermen': 1.5860428231562251,
  'totalhispanicwomen': 0.8081584186079,
  'totalwhite': 1.3728364800364057,
  'totalwomen': 1.188969727674863,
  'totalhispanic': 0.8623332486620477,
  'state': 'WA',
  'totalunknown': 1.1440700983198298,
  'totalasianmen': 1.5969621306699957,
  'totalblackmen': 0.5635375945575468,
  'totalhispanicmen': 0.9533985370263829,
  'totalindianwomen': 2.2614514826352834,
  'totalunknownmen': 1.2474546987803121,
  'totalislanderwomen': 1.4577259475218658,
  'totalmen': 1.2448075053438912,
  'totalbiracial': 2.2336735307681512,
  'totalwhitemen': 1.3820253216402827,
  'totalall': 1.212122034717276,
  'totalblackwomen': 0.2519119470855726,


###Calculate the data per state relative to the state
This block will take all the state data and divide it by the totals for each ethnicity in that specific state. This will give us a number for each state which signifies the % of that ethnicity within the state

In [28]:
import copy
#we are going to calculate what % of each state corresponds to each category
#that is to say, we want to map what % of a state out of the total of all states falls into that category
#that way when we draw the map, the colors will be relative to the total
#this is better than drawing the map using the absolute numbers because there is a tendency for
#the absolute enrollment numbers to hide info because the states with the highest enrollmen or 
#states with the most people will always be colored more brightly
data_per_state_relative_to_state = copy.deepcopy(data_per_state)
for state in data_per_state_relative_to_state.keys():
    data_per_state_relative_to_state[state]['totalmen'] = (float(data_per_state_relative_to_state[state]['totalmen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalwomen'] = (float(data_per_state_relative_to_state[state]['totalwomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalindian'] = (float(data_per_state_relative_to_state[state]['totalindian']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalindianmen'] = (float(data_per_state_relative_to_state[state]['totalindianmen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalindianwomen'] = (float(data_per_state_relative_to_state[state]['totalindianwomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalasian'] = (float(data_per_state_relative_to_state[state]['totalasian']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalasianmen'] = (float(data_per_state_relative_to_state[state]['totalasianmen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalasianwomen'] = (float(data_per_state_relative_to_state[state]['totalasianwomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalblack'] = (float(data_per_state_relative_to_state[state]['totalblack']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalblackmen'] = (float(data_per_state_relative_to_state[state]['totalblackmen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalblackwomen'] = (float(data_per_state_relative_to_state[state]['totalblackwomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalhispanic'] = (float(data_per_state_relative_to_state[state]['totalhispanic']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalhispanicmen'] = (float(data_per_state_relative_to_state[state]['totalhispanicmen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalhispanicwomen'] = (float(data_per_state_relative_to_state[state]['totalhispanicwomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalislander'] = (float(data_per_state_relative_to_state[state]['totalislander']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalislandermen'] = (float(data_per_state_relative_to_state[state]['totalislandermen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalislanderwomen'] = (float(data_per_state_relative_to_state[state]['totalislanderwomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalwhite'] = (float(data_per_state_relative_to_state[state]['totalwhite']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalwhitemen'] = (float(data_per_state_relative_to_state[state]['totalwhitemen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalwhitewomen'] = (float(data_per_state_relative_to_state[state]['totalwhitewomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalbiracial'] = (float(data_per_state_relative_to_state[state]['totalbiracial']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalbiracialmen'] = (float(data_per_state_relative_to_state[state]['totalbiracialmen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalbiracialwomen'] = (float(data_per_state_relative_to_state[state]['totalbiracialwomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalunknown'] = (float(data_per_state_relative_to_state[state]['totalunknown']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalunknownmen'] = (float(data_per_state_relative_to_state[state]['totalunknownmen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0 
    data_per_state_relative_to_state[state]['totalunknownwomen'] = (float(data_per_state_relative_to_state[state]['totalunknownwomen']) / data_per_state_relative_to_state[state]['totalall']) * 100.0  
pprint(data_per_state_relative_to_state)

    

{'WA': {'totalasianwomen': 4.837355131291641,
  'totalindianmen': 0.2883377190526846,
  'totalblack': 3.191310676893791,
  'totalunknownwomen': 4.269077879178098,
  'totalbiracialmen': 1.1477520855495214,
  'totalwhitewomen': 34.89446279603606,
  'totalasian': 8.27501259727899,
  'totalislander': 0.26594255640781594,
  'totalislandermen': 0.11197581322434354,
  'totalhispanicwomen': 3.238900397514137,
  'totalwhite': 59.45355803146521,
  'totalwomen': 57.418397626112764,
  'totalhispanic': 5.5120094059683105,
  'state': 'WA',
  'totalunknown': 7.664744415206314,
  'totalasianmen': 3.4376574659873467,
  'totalblackmen': 1.5536644084877667,
  'totalhispanicmen': 2.273109008454174,
  'totalindianwomen': 0.5486814847992834,
  'totalunknownmen': 3.3956665360282177,
  'totalislanderwomen': 0.15396674318347237,
  'totalmen': 42.581602373887236,
  'totalbiracial': 3.1185263982979676,
  'totalwhitemen': 24.559095235429147,
  'totalall': 35722,
  'totalblackwomen': 1.6376462684060245,
  'totalin

###Load data into Pandas dataframes, load folium and create maps
This block will take all the state data and divide it by the totals for each ethnicity in the US. This will give us a number for each state which signifies the % of the total usfor that ethnicity is in that state

In [29]:
from pandas import *
# shove the data into a pandas dataframe
state_data = DataFrame(data=data_per_state.values(), columns=data_per_state.values()[0].keys())
state_data_relative_us = DataFrame(data=data_per_state_relative_to_us.values(), columns=data_per_state_relative_to_us.values()[0].keys())
state_data_relative_state = DataFrame(data=data_per_state_relative_to_state.values(), columns=data_per_state_relative_to_state.values()[0].keys())


In [30]:
import folium
from IPython.display import HTML
state_geo = 'us_state.json'
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))



### Enrollment for Men as a % of total US enrollments, then as a % of total within the state

In [31]:
map1 = folium.Map(location=[48, -102], zoom_start=3)
map1.geo_json(geo_path=state_geo, data=state_data_relative_us,
             columns=['state', 'totalmen'],
             data_out='men_us.json',
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Men in Graduate Programs as a % of total enrollments')
embed_map(map1, path="men_us.html")

In [32]:

map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_state,
             columns=['state', 'totalmen'],
              data_out='men_state.json',
             key_on='feature.id',
             fill_color='YlGnBu', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Men in Graduate Programs as a % of total state enrollments')
embed_map(map2, path="men_state.html")

### Enrollment for women as a % of total US enrollments, then as a % of total within the state

In [33]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_us,
             columns=['state', 'totalwomen'],
              data_out='women_us.json',
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Women in Graduate Programs as a % of total enrollments',
             reset = True)
embed_map(map2, path="women_us.html")

In [34]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_state,
             columns=['state', 'totalwomen'],
             data_out='women_state.json',
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Women in Graduate Programs as a % of total state enrollments',
             reset = True)
embed_map(map2, path="women_state.html")

### Enrollment for Asians as a % of total US enrollments, then as a % of total within the state

In [35]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_us,
             columns=['state', 'totalasian'],
             data_out='asians_us.json',
             key_on='feature.id',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Asians in Graduate Programs as a % of total enrollments',
             reset = True)
embed_map(map2, path="asians_us.html")

In [36]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_state,
             columns=['state', 'totalasian'],
             key_on='feature.id',
              data_out='asains_state.json',
             fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Asians in Graduate Programs as a % of total state enrollments',
             reset = True)
embed_map(map2, path="asians_state.html")

### Enrollment for Blacks/African Americans as a % of total US enrollments, then as a % of total within the state

In [37]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_us,
             columns=['state', 'totalblack'],
              data_out='black_us.json',
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Blacks/African Americans in Graduate Programs as a % of total enrollments',
             reset = True)
embed_map(map2, path="black_us.html")

In [38]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_state,
             columns=['state', 'totalblack'],
              data_out='black_state.json',
             key_on='feature.id',
             fill_color='YlGn', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Blacks/African Americans in Graduate Programs as a % of total state enrollments',
             reset = True)
embed_map(map2, path="black_state.html")

### Enrollment for Hispanics as a % of total US enrollments, then as a % of total within the state

In [39]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_us,
             columns=['state', 'totalhispanic'],
              data_out='hispanic_us.json',
             key_on='feature.id',
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Hispanics in Graduate Programs as a % of total enrollments',
             reset = True)
embed_map(map2, path="hispanic_us.html")

In [40]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_state,
             columns=['state', 'totalhispanic'],
              data_out='hispanic_state.json',
             key_on='feature.id',
             threshold_scale=[1, 4, 5,7,15],
             fill_color='BuPu', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for Hispanics in Graduate Programs as a % of total state enrollments',
             reset = True)
embed_map(map2, path="hispanic_state.html")

### Enrollment for Pacific Islanders as a % of total US enrollments, then as a % of total within the state

In [41]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_us,
             columns=['state', 'totalislander'],
              data_out='islander_us.json',
             key_on='feature.id',
             threshold_scale=[0, 1, 2, 3, 5],
             fill_color='YlGnBu', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for pacific islanders in Graduate Programs as a % of total enrollments',
             reset = True)
embed_map(map2, path="islander_us.html")

In [42]:
map2 = folium.Map(location=[48, -102], zoom_start=3)
map2.geo_json(geo_path=state_geo, data=state_data_relative_state,
             columns=['state', 'totalislander'],
              data_out='islander_state.json',
             key_on='feature.id',
             threshold_scale=[0, .25, .5, 1],
             fill_color='YlGnBu', fill_opacity=0.7, line_opacity=0.3,
             legend_name='Enrollment for pacific islanders in Graduate Programs as a % of total enrollments',
             reset = True)
embed_map(map2, path="islander_state.html")